In [1]:
from utils import crawl_wayback, create_csv, get_urls

In [ ]:
articles=crawl_wayback("https://www.washingtontimes.com/news/politics/?page=1", 30, get_urls, '20221228', ['article'])
create_csv(articles, 'testing', "test.csv",)

In [1]:
# Util Functions
import time
import requests
from urllib.parse import urlparse
import lxml.html
from wayback import WaybackClient, memento_url_data, WaybackSession
import itertools
import datetime
from utils import make_request, parse_html, make_link_absolute, page_grab 
from crawler import Fox
    
a=Fox()
a.crawl([2022,1,1],[2022,1,10],4)

2022-01-01 00:00:00+00:00 next 2022-01-10 00:00:00+00:00
Fetching https://web.archive.org/web/20220101040818/http://foxnews.com/politics
2022-01-01 04:08:18+00:00 next 2022-01-10 00:00:00+00:00
2022-01-01 08:08:18+00:00 next 2022-01-10 00:00:00+00:00
Fetching https://web.archive.org/web/20220102024844/https://www.foxnews.com/politics
2022-01-02 02:48:45+00:00 next 2022-01-10 00:00:00+00:00
Fetching https://web.archive.org/web/20220102024845/http://foxnews.com/politics
2022-01-03 01:32:31+00:00 next 2022-01-10 00:00:00+00:00
Fetching https://web.archive.org/web/20220103013231/https://www.foxnews.com/politics
2022-01-03 05:32:31+00:00 next 2022-01-10 00:00:00+00:00
Fetching https://web.archive.org/web/20220104001025/http://foxnews.com/politics
2022-01-04 00:10:25+00:00 next 2022-01-10 00:00:00+00:00
2022-01-04 04:10:25+00:00 next 2022-01-10 00:00:00+00:00
Fetching https://web.archive.org/web/20220105002000/https://www.foxnews.com/politics
2022-01-05 00:48:09+00:00 next 2022-01-10 00:00:0

{'http://foxnews.com/health/fauci-milder-covid-not-signal-less-restrictions',
 'http://foxnews.com/health/states-scramble-covid-tests-biden-2022-rapid-test',
 'http://foxnews.com/lifestyle/christmas-code-of-vets-gives-back-veterans-in-need',
 'http://foxnews.com/lifestyle/surfing-santas-florida-space-coast-12th-annual',
 'http://foxnews.com/media/biden-bow-out-democrats-alternative-kurtz-media-buzz',
 'http://foxnews.com/media/cnn-jeffrey-toobin-merrick-garland-jan-6',
 'http://foxnews.com/media/elizabeth-holmes-banning-conservatives-silicon-valley-kurtz-media-buzz',
 'http://foxnews.com/media/jeff-van-drew-former-democrat-urges-manchin-democratic-affiliation',
 'http://foxnews.com/media/kayleigh-mcenany-aoc-response-critics-florida-bar-maskless',
 'http://foxnews.com/media/newt-gingrich-predicts-replay-of-2021-with-disastrous-results-for-democrats',
 'http://foxnews.com/media/one-year-later-why-january-6th-still-erodes-our-democracy',
 'http://foxnews.com/media/pandemic-fatigue-virus-